In [1]:
from src.utilities.get_stock_data import (
    get_stock_market_data,
    get_industry_stock_mapping_data,
)

In [ ]:
from src.stock_filter import main as stock_filter_main
import asyncio

await stock_filter_main()

In [ ]:
await stock_filter_main()

  0%|          | 0/57 [00:00<?, ?it/s]